## Installation of libraries and packages

In [ ]:
#!pip install xgboost -U
#!pip install librosa -U
''''librosa is the package for music and audio analysis'''
#!pip install hyperopt
'''hyperopt is designed to accommodate Bayesian optimization algorithms based on Gaussian processes and regression tress.'''

## Import libraries

In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)

import os
from tqdm import tqdm
# used to display progress bars which are used in this program
import pickle
'''
pickle is the  process of converting a Python object into a byte stream to store it in a file/database, 
maintain program state across sessions, or transport data over the network.
'''
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import librosa

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV,mutual_info_regression
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.decomposition import PCA

from xgboost import XGBClassifier
'''
eXtreme Gradient Boosting (XGBoost) is a scalable and improved version of the gradient boosting algorithm 
designed for efficacy, computational speed and model performance. 
It is an open-source library and a part of the Distributed Machine Learning Community. 
XGBoost is a perfect blend of software and hardware capabilities designed to enhance existing boosting techniques with accuracy 
in the shortest amount of time.
'''
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
#For hyperparameter tuning

## Load the dataset

In [ ]:
'''
# we are only considering 30 seconds files 
because it is still difficult to determine the genre of music based off of 3 seconds.
'''
df = pd.read_csv('Data/features_30_sec.csv')
df.head()

## Exploring the dataset

In [ ]:
df.label.value_counts()
# to check how many genres are present and how many songs each genre has

df.info()
df.describe()
'''
Only 1000 songs are in the dataset with no null values
length is not a relevant variable which needs to be dropped in model training phase
There are some other music features that this dataset did not extract, 
for example, spectral contrast, spectral flatness, tonal centroid features etc. 
Given it is just a small dataset, we can extract them and put it in the dataframe for further analysis
'''

## Extracting extra features and data cleaning

In [ ]:
songs_path = 'Data/genres_original'

# code to extract the 4 additional features from the 1000 music files present in the dataset.
def extract_new_features(song_path, num_files = 1000, num_new_features = 8):
    data_array = np.empty([num_files, num_new_features])
    counter = 0
    for root, dirs, files in os.walk(songs_path):
        dirs.sort()
        for file, i in zip(sorted(files), tqdm(range(num_files))):
            i = i + (counter-1)*100
            file_path = os.path.join(root, file)
        
            try:
                #extract mean and variance of those 4 features
                y, sr = librosa.load(os.fspath(file_path))
                chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
                spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
                spectral_flatness = librosa.feature.spectral_flatness(y=y)
                tonnetz = librosa.feature.tonnetz(y=y, sr=sr)

                data_array[i,0] = np.mean(chroma_cens)
                data_array[i,1] = np.var(chroma_cens)
                data_array[i,2] = np.mean(spectral_contrast)
                data_array[i,3] = np.var(spectral_contrast)
                data_array[i,4] = np.mean(spectral_flatness)
                data_array[i,5] = np.var(spectral_flatness)
                data_array[i,6] = np.mean(tonnetz)
                data_array[i,7] = np.var(tonnetz)
                
            # Set all values to zero for files with problems
            except:
                print(f'Problem file: {file_path}')
                for j in range(num_new_features):
                    data_array[i, j] = 0 
                
        counter += 1
                  
    return data_array


new_features_array = extract_new_features('Data/genres_original')

## Adding the above extracted 4 features to the already selected 4 features which are present in the dataframe

In [ ]:
df['chroma_cens_mean'] = new_features_array[:,0]
df['chroma_cens_var'] = new_features_array[:,1]
df['spectral_contrast_mean'] = new_features_array[:,2]
df['spectral_contrast_var'] = new_features_array[:,3]
df['spectral_flatness_mean'] = new_features_array[:,4]
df['spectral_flatness_var'] = new_features_array[:,5]
df['tonnetz_mean'] = new_features_array[:,6]
df['tonnetz_var'] = new_features_array[:,7]
'''
Since there is only 1 row of missing value, 
now we'll just fill in the missing values of jazz.0054.wav as the mean of those of the jazz songs, which is at index 554
'''

## Filtering out the jazz genre expect for jazz.0054

In [ ]:
for i in range(-8,0,1):
    # Filter out the jazz genre except jazz.0054
    df.iloc[554,i] = df[ df.label =='jazz'].iloc[np.r_[np.arange(0,54),np.arange(55,100)],i].mean()

df.to_csv('new_csv',index=False)
# saving the index without the index column which comes by default

df = df.iloc[:,2:]

## EDA and Data Preprocessing

In [ ]:
corr = df.corr()

#Create a mask for the heatmap
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

plt.subplots(figsize=(20, 20))
sns.heatmap(corr, mask=mask, cmap="vlag")
''''
Most of the variables do not have a high correlation with other variables. 
Let's filter out the extremely highly correlated pairs and examine them.
'''

sol = (corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
                  .stack()
                  .sort_values(ascending=False))
for index, value in sol.items():
    if (value > 0.75) or (value < -0.75):
        print(index, value)
'''
using this code we are able to extract the features which are correlated with each other and 
also by how much from the above map
'''

## Splitting train and test data

In [ ]:
'''Given the small number of training data, 
We set 90% as training data and 10% as testing data. 
For hyparameter tuning, given the small dataset, we will use the same train dataset to tune For the split of data, 
We made sure every class has the same number of data to train and test.
'''
y = df.label
X = df

#Use `label` to split data evenly and drop `label` column after split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=df.label, random_state=77)
X_train.drop('label',axis=1,inplace=True)
X_test.drop('label',axis=1,inplace=True)

## Normalize the data

In [ ]:
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
X_test_scaled = sc.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)

## Initial model fitting and recursive feature elimination 

In [ ]:
'''
Now we will fit our training data to xgboost classifier first, and 
then we’ll do RFECV to check which variables can be eliminated
RFECV stands for Recursive Feature Elimination and Cross-Validation Selection 
which does Backward selection in Python: Scikit-Learn.
'''
estimator = XGBClassifier(eval_metric='merror')
rfecv = RFECV(estimator, step=1, cv=5,scoring='accuracy',verbose=1)
rfecv.fit(X_train, y_train)

# if step > 1 then step corresponds to the (integer) number of features to remove at each iteration.

## Checking which features can be further deleted

In [ ]:
features_drop_array = list(np.where(rfecv.support_ == False)[0])
X_train.columns[features_drop_array]

X_train.drop(X_train.columns[features_drop_array], axis=1, inplace=True)
X_test.drop(X_test.columns[features_drop_array], axis=1, inplace=True)

# this code determines which columns can be eliminated and drops them, this happens in both train and test data

## Model training

In [ ]:
model = XGBClassifier(n_estimators=1000)
model.fit(X_train,y_train,eval_metric='merror')
'''
eval_metric is the evaluation metric for validation data, 
a default metric will be assigned according to objective (rmse for regression, and logloss for classification) 
but here we use rmse -> root mean square error for better yielding results
'''

## Getting the accuracy for both train and test data

In [ ]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
target_names = sorted(set(y))

print(f'Training accuracy: {accuracy_score(y_train,y_pred_train)}')
print(f'Training:\n {classification_report(y_train, y_pred_train, labels=target_names)}')
print(f'Testing accuracy: {accuracy_score(y_test,y_pred_test)}')
print(f'Testing:\n {classification_report(y_test, y_pred_test, labels=target_names)}')

#Confusion matrix of the test data
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize = (16, 9))
sns.heatmap(cm,cmap="Blues", annot=True, xticklabels = target_names, yticklabels = target_names )

'''
this will display training accuracy as 0.99 while test data is 0.78 
which indicates that the model has been overfit. 
Hence, we will add regularization parameters and tune other parameters to reduce this problem.
'''

## Hyperparameter tuning

In [ ]:
'''
When creating a machine learning model, you'll be presented with design choices as to how to define your model architecture. 
Often times, we don't immediately know what the optimal model architecture should be for a given model, 
and thus we'd like to be able to explore a range of possibilities. 
In true machine learning fashion, we'll ideally ask the machine to perform this exploration and 
select the optimal model architecture automatically. 
Parameters which define the model architecture are referred to as hyperparameters and 
thus this process of searching for the ideal model architecture is referred to as hyperparameter tuning.
'''
# In the following, we'll use hyperopt library to help tuning the parameters. The parameters may vary for each run

space={
    'n_estimators': hp.quniform('n_estimators', 0,3000,1),
    'reg_lambda' : hp.quniform('reg_lambda', 0,500,1),
    }

def objective(space):
    clf=XGBClassifier(
                    n_estimators =int(space['n_estimators']),
                    reg_lambda = int(space['reg_lambda']),
                    )
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    return {'loss': -accuracy, 'status': STATUS_OK }


trials = Trials()
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

print(f"best params: {best_hyperparams}")

# this will output the best n_estimators and reg_lambda values to be used for the model

## Running the model again after hyperparameter tuning

In [ ]:
model1 = XGBClassifier(n_estimators=304, reg_lambda=25)
model1.fit(X_train,y_train,eval_metric='merror')
y_pred_test1 = model1.predict(X_test)
print(f"accuracy: {accuracy_score(y_test,y_pred_test1)}")
print(f'New tuned model:\n {classification_report(y_test, y_pred_test1, labels=target_names)}')

## Save the model and the preprocessing

In [ ]:
pickle.dump(sc, open('sc.pkl','wb'))
pickle.dump(model1, open('model.pkl', 'wb'))